# Introduction to Code Generation

This notebook exemplifies how to generate automatically code in terms of interfaces. For this three examples are provided. The first one creates a ROS message, the second a standard python class and the last a protobuf interface.

In [1]:
import capellambse

In [2]:
path_to_model = "../tests/data/melodymodel/5_0/Melody Model Test.aird"
model = capellambse.MelodyModel(path_to_model)
model

Cannot load PVMT extension: ValueError: Provided model does not have a PropertyValuePkg
Property values are not available in this model


In particular, we want to create code from our class:

In [3]:
model.diagrams.by_name("[CDB] CodeGeneration")

Unknown global filter 'hide.association.labels.filter'
Unknown global filter 'hide.technical.interfaces.filter'
Unknown global filter 'ModelExtensionFilter'
No default style for 'Box.NumericType' in 'Class Diagram Blank'
No default style for 'Box.NumericType' in 'Class Diagram Blank'
No default style for 'Box.NumericType' in 'Class Diagram Blank'
No default style for 'Box.NumericType' in 'Class Diagram Blank'
No default style for 'Box.NumericType' in 'Class Diagram Blank'
No default style for 'Box.NumericType' in 'Class Diagram Blank'
No default style for 'Box.NumericType' in 'Class Diagram Blank'
No default style for 'Box.StringType' in 'Class Diagram Blank'
No default style for 'Box.StringType' in 'Class Diagram Blank'
No default style for 'Box.StringType' in 'Class Diagram Blank'
No default style for 'Box.StringType' in 'Class Diagram Blank'
No default style for 'Box.StringType' in 'Class Diagram Blank'
No default style for 'Box.StringType' in 'Class Diagram Blank'
No default style 

In order to access to the classes, we simply can access to _data_package_ of the operational layer and from there to the attribute _classes_.

In [4]:
data_pkg = model.oa.data_package
data_pkg

classes,"Class ""Twist"" (8164ae8b-36d5-4502-a184-5ec064db4ec3)Class ""Trajectory"" (16fc56f3-d86e-4c13-b2fe-10a498c0b6d5)Class ""Waypoint"" (1b84a3be-197c-41cd-aa95-d27b0bf2ce58)Class ""Example"" (f0dc5e0c-caa6-42de-979c-acf9436183bf)"
collections,(Empty list)
complex_values,(Empty list)
constraints,(Empty list)
description,None
diagrams,[CDB] Data (uuid: _beibwNYrEeqiU8uzTY0Puw)[CDB] CodeGeneration (uuid: _PBVPQLQiEeysLP7qUEj5NQ)
enumerations,(Empty list)
name,Data
packages,(Empty list)
parent,"OperationalAnalysis ""Operational Analysis"" (ddbef16d-ddb9-4162-934c-f1e40e6f8bed)"
progress_status,NOT_SET


# ROS2 IDL Message

Let's have a brief looj into the structure of ROS2 Message descriptions. They are stored in .msg files and comprises two parts: type and name, separated by a space, i.e.:

### _class.msg_

fieldtype1 fieldname1

fieldtype2[] fieldname2

In [5]:
def class_to_ros2_idl(cls):
    filename = f"{cls.name}.msg"
    text = []
    for prop in cls.properties:
        multiplicity = ("", "[]")[prop.max_card.value > 1.0]
        text.append(f"{prop.type.name}{multiplicity} {prop.name}")
    text = "\n".join(text)
    with open(filename, "w") as file:
        file.write(text)
    print(f"# file: {filename} \n{text}\n")

In our example, files would be generated with the following content:

In [6]:
data_pkg = model.oa.data_package
for cls in data_pkg.classes:
    class_to_ros2_idl(cls)

# file: Twist.msg 


# file: Trajectory.msg 
Waypoint[] waypoints

# file: Waypoint.msg 
float lat
float lon
float alt
Example[] examples

# file: Example.msg 
str test



# Interface for python class

A python class has the following structure:
### _class.py_

class class_name:
> name1: type  
> name2: [type]

A python interface can be generated as follows:

In [7]:
def class_to_python(cls, current_classes=None):
    text = [f"class {cls.name}:"]
    current_classes = [cls]
    if not cls.properties:
        text.append("\tpass")
    for prop in cls.properties:
        if (
            isinstance(
                prop.type, capellambse.model.crosslayer.information.Class
            )
            and prop.type not in current_classes
        ):
            nested_text = class_to_python(prop.type, current_classes)
            text = [nested_text] + ["\n"] + text
        multiplicity = (f"{prop.type.name}", f"list[{prop.type.name}]")[
            prop.max_card.value > 1.0
        ]
        text.append(f"\t{prop.name}: {multiplicity}")
    text = "\n".join(text)

    return text

In [8]:
trajectory = data_pkg.classes.by_name("Trajectory")
text = class_to_python(trajectory)
filename = f"{trajectory.name}.py"
with open(filename, "w") as file:
    file.write(text)
print(f"# file: {filename} \n{text}\n")

# file: Trajectory.py 
class Example:
	test: str


class Waypoint:
	lat: float
	lon: float
	alt: float
	examples: list[Example]


class Trajectory:
	waypoints: list[Waypoint]



# Interface for Protocol Buffers (Protobuf) 

Protobuf Message descriptions are stored in .proto files where class defintion starts with _message_ and each property of the class is definded by at least three parts: the data type, name and their order number. Classes can be also nested in classes. An example is shown In the following:

### _class.proto_

syntax = "proto2";

message class1 {

>    datatype class1_name1 = 1;

>    dataype class1_name2 = 2;

>    message class2 {

> >  datatype class2_name1 = 1;

> >  datatype class2_name2 = 2;

> >  datatype class2_name3 = 3;

 >   }

>    repeated class2 class_name = 3;
    
}


In [9]:
def class_to_proto(cls, current_classes=[], indent="\t"):
    if not current_classes:
        text = ['syntax = "proto3";\n']
    text = [f"{indent[:-1]}message  {cls.name} {{"]
    current_classes.append([cls])

    for counter, prop in enumerate(cls.properties):
        multiplicity = ("", "[]")[prop.max_card.value > 1.0]
        if (
            isinstance(
                prop.type, capellambse.model.crosslayer.information.Class
            )
            and prop.type not in current_classes
        ):
            current_classes.append([prop.type])
            nested_text = class_to_proto(
                prop.type, current_classes, indent + "\t"
            )
            text.append(nested_text)
            text.append(
                f"{indent}repeated {prop.type.name}{multiplicity} {prop.name} = {counter+1};"
            )
        else:
            text.append(
                f"{indent}{prop.type.name}{multiplicity} {prop.name} = {counter+1};"
            )
    text[-1] = text[-1] + f"\n{indent[:-1]}}}"
    text = "\n".join(text)
    return text

The protobuf interface of class _Trajectory_ would look as follows:


In [10]:
trajectory = data_pkg.classes.by_name("Trajectory")
text = class_to_proto(trajectory)
filename = f"{trajectory.name}.proto"
with open(filename, "w") as file:
    file.write(text)
print(f"# file: {filename} \n{text}\n")

# file: Trajectory.proto 
message  Trajectory {
	message  Waypoint {
		float lat = 1;
		float lon = 2;
		float alt = 3;
		message  Example {
			str test = 1;
		}
		repeated Example[] examples = 4;
	}
	repeated Waypoint[] waypoints = 1;
}

